<a href="https://colab.research.google.com/github/Jasper-Hewitt/Wind_project/blob/main/extra_finding_noisepollution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#this one uses a different regex, split on 。 and ：
#many of the units of information within the PDF are actually only one sentence so I guess this is fine. 
!pip install fitz
!pip install static
!pip install frontend
!pip install pymupdf --upgrade
!pip install openai

In [2]:
# mkdir static
import fitz

doc = fitz.open("dragged_15_37.pdf")

for page in doc:
    text = page.get_text("text")
    with open("output4.txt", "a") as f:
        f.write(text)

In [ ]:
with open('output4.txt', 'r') as file:
    # Read the contents of the file
    contents = file.read()
    # Print the contents
    print(contents)

In [5]:
import re
pattern = "。|:"
split_contents = re.split(pattern, contents)
# print(sentences)

In [6]:
split_contents

['定稿本修正��對照表 \n \n \n1 \n大彰化東北離岸風力發電計畫環境影響說明書 \n第三次修訂本及定稿本 修正前後對照表 \n第三次修訂本 \n定稿本 \n修正說明 \n審查結論公告隔頁紙 \n審查結論公告 \n[環署綜字第 1070023183B 號函] \n審查結論公告隔頁紙 \n行政院環境保護署函文 \n環署綜字第 1070023183B 號函 \n審查結論公告 \n環署綜字第 1070023183 號函公告 \n依據「大彰化東北離岸\n風力發電計畫環境影響\n說明書」（定稿本）修正\n意見第三條，重新確認\n「審查結論公告」分頁\n標題與所載文號',
 ' \np4-1~2 \n \n圖 4.2-1  大彰化東北離岸風力發電計畫位置圖 \n \n圖 4.2-1  大彰化東北離岸風力發電計畫位置圖(續) \np4-1 \n \n圖 4.2-1  大彰化東北離岸風力發電計畫位置圖(1/2) \n \n圖 4.2-1  大彰化東北離岸風力發電計畫位置圖(2/2) \n依據「大彰化東北離岸\n風力發電計畫環境影響\n說明書」（定稿本）修正\n意見第五條，刪除原規\n劃於彰化縣線西鄉上岸\n等相關圖說，更新為二\n萬五千分之一地形圖',
 ' \n \n \n2 \np.4-3 \n1.離岸風場海域：本計畫風場原面積約 111.8 平方公里，依\n交通部航港局 106 年 8 月 11 日公告之預定航道座標修正後\n面積約 108.2 平方公里，海域水深介於 34~44 公尺，風機單\n機裝置容量介於 4~11MW',
 '當選用單機裝置容量最小(4MW)\n的風機時，將設置 140 部風機；當選用單機裝置容量最大\n(11MW)的風機時，將設置 51 部風機，總裝置容量不超過\n570MW',
 ' \np.4-2 \n1.離岸風場海域：本計畫風場原面積約 111.8 平方公里，依交\n通部航港局 106 年 8 月 11 日公告之預定航道座標修正後面積\n約 108.2 平方公里，海域水深介於 34~44 公尺，風機單機裝\n置容量介於 8~11MW',
 '當選用單機裝置容量最小(8MW)的風\n機時，將設置 71 部風機；當選用單機裝置容量最大(11MW)\n的風機時，將設置 51 部風機，最大總裝置容量不大於\n570MW',
 ' \n依第 3

In [9]:
len(split_contents)

222

In [ ]:
# import re
# split_contents = re.split(r'\d+\.', contents)
# split_contents

In [10]:
!pip install sentence_transformers
#also works with entire alineas 
#below are some other models that don't work. I get a warning signal that says it can't find the model so it switches to MEAN pooling instead. 
#('all-MiniLM-L6-v2'), one of the most popular ones, works but it is an English model 
#("uer/sbert-base-chinese-nli")#('shibing624/text2vec-base-chinese')#('all-MiniLM-L6-v2') 

from sentence_transformers import SentenceTransformer, util
import torch

#alternative model by same creators: ("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"). This one appears to be more popular. Compare results.
# embedder = SentenceTransformer("distiluse-base-multilingual-cased-v1")
embedder = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
# Corpus with example sentences
corpus = split_contents
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# Query sentences:
queries = ['噪音', '水下噪音','聲音']

# Find the closest X sentences of the corpus for each query sentence based on cosine similarity
top_k = min(68, len(corpus))
# for query in queries:
#     query_embedding = embedder.encode(query, convert_to_tensor=True)

#     # We use cosine-similarity and torch.topk to find the highest 5 scores
#     cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
#     top_results = torch.topk(cos_scores, k=top_k)

#     print("\n\n======================\n\n")
#     print("Query:", query)
#     print("\nTop X most similar sentences in corpus:")

#     for score, idx in zip(top_results[0], top_results[1]):
#         print(corpus[idx], "(Score: {:.4f})".format(score))

for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop X most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print("Sentence: ", corpus[idx], " Original Sequence: ", idx+1, " (Score: {:.4f})".format(score))


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 100.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 80.3 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=0b9073d07b58c2419860a4818760e9ac05e3e9a78a3e3181ede3db8846cb426c
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers






Query: 噪音

Top X most similar sentences in corpus:
Sentence:   
  Original Sequence:  tensor(222)  (Score: 0.5277)
Sentence:  … 
修正報告內用字，將「我
國」、「台灣」一詞統一
修正為「臺灣」，以利閱
讀  Original Sequence:  tensor(151)  (Score: 0.2434)
Sentence:  … 
修正報告內用字，補充
「ROV」一詞之中英文
全稱及簡稱，以利閱讀  Original Sequence:  tensor(130)  (Score: 0.2284)
Sentence:   
… 
修正報告內用字，補充
「ROV」一詞之中英文
全稱及簡稱，以利閱讀  Original Sequence:  tensor(127)  (Score: 0.2284)
Sentence:  定稿本修正��對照表 
 
 
1 
大彰化東北離岸風力發電計畫環境影響說明書 
第三次修訂本及定稿本 修正前後對照表 
第三次修訂本 
定稿本 
修正說明 
審查結論公告隔頁紙 
審查結論公告 
[環署綜字第 1070023183B 號函] 
審查結論公告隔頁紙 
行政院環境保護署函文 
環署綜字第 1070023183B 號函 
審查結論公告 
環署綜字第 1070023183 號函公告 
依據「大彰化東北離岸
風力發電計畫環境影響
說明書」（定稿本）修正
意見第三條，重新確認
「審查結論公告」分頁
標題與所載文號  Original Sequence:  tensor(1)  (Score: 0.2235)
Sentence:  
 
 
表 6.1-1 開發行為可能影響範圍之各種相關計畫(續 2) 
範圍
計畫名稱 主管單位 完成時間
相  互  關  係  或  影  響 
發
行
為
沿
線
兩
側
各
百
公
尺
範
圍
內
開
發
行
為
半
徑
十
公
里
範
圍
內
或
線
型
式
 
福 海 離 岸
風 力 發 電
計畫(第一
期工程) 
經濟部
能源局
109年
該計畫於彰化縣芳苑鄉西側海域距岸約8
公里處設置2座離岸風機及1座海氣象觀測
塔，與本計畫皆是以風力發電方式，對臺
灣電力供應及穩定性皆有

## calculate accumulated similarity score per keyword

In [11]:
sentence_scores = {sentence: {query: 0 for query in queries} for sentence in corpus}

for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=len(corpus))
    for score, idx in zip(top_results[0], top_results[1]):
        sentence = corpus[idx]
        sentence_scores[sentence][query] = score

In [12]:
len(sentence_scores)

202

In [13]:
import pandas as pd
#in case we use more or less keywords, we will have to adjust this code slightly
# df = pd.DataFrame([[sentence, scores[queries[0]], scores[queries[1]], 
#                     scores[queries[2]], scores[queries[0]]+scores[queries[1]]+scores[queries[2]]] 
#                    for sentence, scores in sentence_scores.items()], 
#                   columns=['text', 'similarity_score1', 'similarity_score2', 'similarity_score3', 
#                            'accumulated_score'])
# df = df.sort_values(by='accumulated_score', ascending=False)

results = []
for i, (sentence, scores) in enumerate(sentence_scores.items()):
    results.append((sentence, i+1, scores[queries[0]], scores[queries[1]], scores[queries[2]], scores[queries[0]]+scores[queries[1]]+scores[queries[2]]))

df = pd.DataFrame(results, columns=['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'accumulated_score'])
df = df.sort_values(by='accumulated_score', ascending=False)


In [14]:
noise_pattern= '遵照辦理'
noise_pattern1='章節'
noise_pattern2='��'
noise_pattern3='�'
noise_pattern4='修訂處'
noise_pattern5='頁次'

df['text'] = df['text'].replace(noise_pattern,'',regex=True)
df['text'] = df['text'].replace(noise_pattern1,'',regex=True)
df['text'] = df['text'].replace(noise_pattern2,'',regex=True)
df['text'] = df['text'].replace(noise_pattern3,'',regex=True)
df['text'] = df['text'].replace(noise_pattern4,'',regex=True)
df['text'] = df['text'].replace(noise_pattern5,'',regex=True)

In [15]:
#delete empty rows
df = df[df['text'].str.len()>6]   
df = df[['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'accumulated_score']]
df = df.reset_index(drop=True)
df

,text,original_sequence,similarity_score1,similarity_score2,similarity_score3,accumulated_score
0,非必要\np5-33 \n三、風機基礎除役：… \n… \n(二) 使用無人水下載具(Rem...,109,tensor(0.1882),tensor(0.4254),tensor(0.2218),tensor(0.8354)
1,… \n修正報告內用字，補充\n「ROV」一詞之中英文\n全稱及簡稱，以利閱讀,118,tensor(0.2284),tensor(0.1852),tensor(0.3139),tensor(0.7276)
2,\n… \n修正報告內用字，補充\n「ROV」一詞之中英文\n全稱及簡稱，以利閱讀,115,tensor(0.2284),tensor(0.1852),tensor(0.3139),tensor(0.7276)
3,… \n修正報告內用字，將「我\n國」、「台灣」一詞統一\n修正為「臺灣」，以利閱\n讀,137,tensor(0.2434),tensor(0.1809),tensor(0.2970),tensor(0.7213)
4,\n… \np5-34 \n五、離岸變電站：… \n… \n(四) 吊放工作級無人水下載具...,114,tensor(0.1327),tensor(0.3918),tensor(0.1848),tensor(0.7094)
...,...,...,...,...,...,...
196,\n永續能源政\n策綱領 \n經濟\n部\n114\n年 \n本計畫依循政府提高再生能源利...,147,tensor(-0.0622),tensor(-0.0525),tensor(-0.0472),tensor(-0.1620)
197,\n再生能源發\n展條例 \n經濟部\n－ \n本計畫於該條例保障下，未來生產電\n力將併...,144,tensor(-0.0678),tensor(-0.0559),tensor(-0.0811),tensor(-0.2048)
198,\n再生能源發\n展條例 \n經濟\n部\n－ \n本計畫於該條例保障下，未來生產電力將併...,150,tensor(-0.0819),tensor(-0.0616),tensor(-0.0883),tensor(-0.2318)
199,\np5-34 \n本籌備處屬沃旭能源公司旗下子公司，沃旭能源公司於臺灣\n共計申請四個潛力...,121,tensor(-0.0921),tensor(-0.0614),tensor(-0.0984),tensor(-0.2519)


## determine threshold

In [16]:
#we will have to consider how we want to determine the threshold, the simplest way would be to look at the dataframe and check from where it doesn't have anything to do 
#with 噪音 anymore. if there is a more mathematical way to solve this problem I am also open to that. 
threshold = 0.7710
mask = df['accumulated_score'].apply(lambda x: x.item() > threshold)

sbert_out = df[mask]
sbert_out

,text,original_sequence,similarity_score1,similarity_score2,similarity_score3,accumulated_score
0,非必要\np5-33 \n三、風機基礎除役：… \n… \n(二) 使用無人水下載具(Rem...,109,tensor(0.1882),tensor(0.4254),tensor(0.2218),tensor(0.8354)


## sort by original sequence (optional)

In [ ]:
# sbert_out = sbert_out.sort_values(by='original_sequence').reset_index(drop=True)
# sbert_out=sbert_out['text']

In [ ]:
# sbert_out=sbert_out.to_frame()
# sbert_out

## Summary with GPT 

In [ ]:
#disable this cell for sort by orginal sequence 
sbert_out=sbert_out['text']  
sbert_out=sbert_out.to_frame() 
sbert_out

In [ ]:
sbert_out.loc[0] = ['幫我寫總結之下的內容，強調關於噪音的規格的改變，請你使用比較簡單的語言，小孩子也可以看得懂']
# sbert_out=sbert_out.to_frame()

In [ ]:
text = sbert_out['text'].to_string(index=False)

In [ ]:
text = re.sub(r'[\n\\]+', '', text)
text= re.sub(r'n|n－', '', text)
text

In [ ]:
#there is one problem with chatgpt and that is that it only allows us to insert 4000 tokens. so we either have to be super selective with the information we put in, or 
#put it in in batches. 
#I think it counts whitespaces as tokens. because when I copy paste this in word it says it is only 923 words. try a regex that deletes whitespaces ands see what happens.
import os
import openai
import json

openai.api_key = '' #remove key when uploading to github
summary = ''

response = openai.Completion.create(
  model="text-davinci-003",
  prompt= text,
  temperature=0.9,
  max_tokens=1000,
  top_p=1,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=[" Human:", " AI:"]
  )
response
json_object = json.loads(str(response))
text = json_object['choices'][0]['text']
summary = summary + text

summary

In [ ]:
#output with ordering the sentences
#等，以發揮學生創意積極參與改善n潮州沿海之環境素質等，降低潮州沿海n噪音污染等之危害，以便傳達對環境維護n及發展之重聯 。
#\n\n總結：本計畫根據第327次環境影響評估審查委員會的決議，於施工期間規劃了水下噪音檢測以及禁用聲音驅離裝置，嚴格遵守噪音振動模擬結果表明的規定，
#並且將於距離打樁位置750公尺外選擇合理方位執行，達到減低潮州沿海噪音污染等之危害，並且協助指導學生參與改善環境素質的目的。